In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy

In [85]:
train_data = pd.read_csv("../data/mnist_train.csv")
test_data = pd.read_csv("../data/mnist_test.csv")

In [86]:
train_data_y = train_data.label
train_data_x = train_data.drop(columns=['label'])

input_train_data_x = np.asfarray(train_data_x)/255.0
input_train_data_x = np.array(input_train_data_x, ndmin=2).T
input_train_data_y = np.zeros((train_data_y.shape[0], 10))
input_train_data_y[np.arange(train_data_y.shape[0]), train_data_y] = 1
input_train_data_y = np.array(input_train_data_y, ndmin=2).T

In [95]:
input_train_data_y.shape

(10, 60000)

In [87]:
test_data_y = test_data.label
test_data_x = test_data.drop(columns=['label'])

input_test_data_x = np.asfarray(test_data_x)/255.0
input_test_data_x = np.array(input_test_data_x, ndmin=2).T
input_test_data_y = np.zeros((test_data_y.shape[0], 10))
input_test_data_y[np.arange(test_data_y.shape[0]), test_data_y] = 1
input_test_data_y = np.array(input_test_data_y, ndmin=2).T

In [88]:
input_test_data_x.shape

(784, 10000)

In [128]:
def getAccuracy(actualLabels, predictedLabels):
    predicted_digit = np.argmax(predictedLabels, axis=1)
#     actual_digit = np.argmax(actualLabels, axis=1)
    accuracy = np.mean(actualLabels == predicted_digit)
    accuracy_percentage = accuracy * 100
    return accuracy_percentage

In [176]:
class neuralNetwork:
    def __init__(self, inputNodes=784, hiddenNodes1=512, hiddenNodes2=64, outputNodes=10, learningRate=0.00001, epochs=200):
        rng = np.random.default_rng()
        
        self.inputNodes = inputNodes
        self.outputNodes = outputNodes
        self.hiddenNodes1 = hiddenNodes1
        self.hiddenNodes2 = hiddenNodes2
        self.lr = learningRate
        self.epochs = epochs
        
        self.wih1 = np.random.normal(0.0, pow(self.inputNodes, -0.5), (self.hiddenNodes1, self.inputNodes))
        self.wh1h2 = np.random.normal(0.0, pow(self.hiddenNodes1, -0.5), (hiddenNodes2, hiddenNodes1))
        self.wh2o = np.random.normal(0.0, pow(self.hiddenNodes2, -0.5), (outputNodes, hiddenNodes2))
        self.activationFunction = sigmoid
        pass
    
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))
    
    def train(self, inputData, outputData):
        for epoch in range(self.epochs):
            inputHidden1 = np.dot(self.wih1, inputData)
            outputHidden1 = self.activationFunction(inputHidden1)

            inputHidden2 = np.dot(self.wh1h2, outputHidden1)
            outputHidden2 = self.activationFunction(inputHidden2)

            inputOutput = np.dot(self.wh2o, outputHidden2)
            outputOutput = self.activationFunction(inputOutput)

            outputError = outputData - outputOutput
            hidden2Error = np.dot(self.wh2o.T, outputError)
            hidden1Error = np.dot(self.wh1h2.T, hidden2Error)

            self.wh2o += self.lr*np.dot((outputError*outputOutput*(1.0-outputOutput)), np.transpose(outputHidden2))
            self.wh1h2 += self.lr*np.dot((hidden2Error*outputHidden2*(1.0-outputHidden2)), np.transpose(outputHidden1))
            self.wih1 += self.lr*np.dot((hidden1Error*outputHidden1*(1.0-outputHidden1)), np.transpose(inputData))
            if (epoch+1) % 5 == 0:
                print("Epoch: " + str(epoch+1))
                print("Total loss for training data is: ", np.mean((outputData - outputOutput) ** 2), " and accuracy is: " + str(getAccuracy(train_data_y, outputOutput.T)))
        
        pass
    
    def predict(self, inputData):
        inputHidden1 = np.dot(self.wih1, inputData)
        outputHidden1 = self.activationFunction(inputHidden1)
        
        inputHidden2 = np.dot(self.wh1h2, outputHidden1)
        outputHidden2 = self.activationFunction(inputHidden2)
        
        inputOutput = np.dot(self.wh2o, outputHidden2)
        outputOutput = self.activationFunction(inputOutput)
        
        return outputOutput

In [177]:
n = neuralNetwork(epochs=200)

In [178]:
n.train(input_train_data_x, input_train_data_y)

Epoch: 5
Total loss for training data is:  0.09012631878178431  and accuracy is: 10.441666666666666
Epoch: 10
Total loss for training data is:  0.08946199448018266  and accuracy is: 16.361666666666665
Epoch: 15
Total loss for training data is:  0.08899987856234784  and accuracy is: 26.321666666666665
Epoch: 20
Total loss for training data is:  0.08837824022395886  and accuracy is: 34.833333333333336
Epoch: 25
Total loss for training data is:  0.08757837356785095  and accuracy is: 40.39666666666666
Epoch: 30
Total loss for training data is:  0.08656809041287364  and accuracy is: 43.22166666666667
Epoch: 35
Total loss for training data is:  0.08529957034879707  and accuracy is: 44.745000000000005
Epoch: 40
Total loss for training data is:  0.08385684206522537  and accuracy is: 47.305
Epoch: 45
Total loss for training data is:  0.08220587111270773  and accuracy is: 49.265
Epoch: 50
Total loss for training data is:  0.08030865286033852  and accuracy is: 51.20666666666666
Epoch: 55
Total lo

In [179]:
predicted = n.predict(input_test_data_x)
predicted.shape

(10, 10000)

In [180]:
getAccuracy(test_data_y, predicted.T)

84.2